In [0]:
from databricks import *
from pyspark.sql.functions import col
import  sys
import os


path = '/Workspace/Users/joshilopez1609@gmail.com/glo-de-chll'

sys.path.append(os.path.abspath(path))

from utils import *

In [0]:
#connection with ADLS


In [0]:
# For testing purposes
dbutils.fs.ls("abfss://raw@storage2de.dfs.core.windows.net/")
dbutils.fs.ls("abfss://backups@storage2de.dfs.core.windows.net/")
dbutils.fs.ls("abfss://logs@storage2de.dfs.core.windows.net/")

In [0]:
dbutils.fs.ls("abfss://raw@storage2de.dfs.core.windows.net/")

In [0]:
#Assigning the schema as needed
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

hired_employees_schema = StructType([
    StructField("id", IntegerType(), True), 
    StructField("name", StringType(), True),  
    StructField("datetime", StringType(), True),  
    StructField("department_id", IntegerType(), True),  
    StructField("job_id", IntegerType(), True)  
])

departments_schema = StructType([
    StructField("id", IntegerType(), True),  
    StructField("department", StringType(), True),  
])


jobs_schema = StructType([
    StructField("id", IntegerType(), True), 
    StructField("job", StringType(), True)  
])


In [0]:
hired_employees_df = spark.read.csv("abfss://raw@storage2de.dfs.core.windows.net/hired_employees.csv", header=False, schema=hired_employees_schema)
departments_df = spark.read.csv("abfss://raw@storage2de.dfs.core.windows.net/departments.csv", header=False, schema=departments_schema)
jobs_df = spark.read.csv("abfss://raw@storage2de.dfs.core.windows.net/jobs.csv", header=False,schema=jobs_schema)

In [0]:
invalid_hired = hired_employees_df.filter(
    col("id").isNull() | col("name").isNull() | col("datetime").isNull() |
    col("department_id").isNull() | col("job_id").isNull()
)
invalid_hired.write.mode("append").csv("abfss://logs@storage2de.dfs.core.windows.net/")
valid_hired = hired_employees_df.na.drop()

invalid_departments = departments_df.filter(
    col("id").isNull() | col("department").isNull()
)
invalid_departments.write.mode("append").csv("abfss://logs@storage2de.dfs.core.windows.net/")
valid_departments = departments_df.na.drop()

invalid_jobs = jobs_df.filter(
    col("id").isNull() | col("job").isNull()
)
invalid_jobs.write.mode("append").csv("abfss://logs@storage2de.dfs.core.windows.net/")
valid_jobs = jobs_df.na.drop()

In [0]:
# Call the function to write in the database

write_to_sql(valid_departments, "departments", jdbc_url, connection_properties)
write_to_sql(valid_jobs, "jobs", jdbc_url, connection_properties)
write_to_sql(valid_hired, "hired_employees", jdbc_url, connection_properties)
